# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_filepath = os.path.join("output_data", "cities.csv")

city_data_df = pd.read_csv(cities_filepath)

city_data_df.head()

,City ID,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,kieta,PG,100,1632165190,79,-6.2167,155.6333,77.43,3.24
1,1,saint-philippe,RE,98,1632165190,82,-21.3585,55.7679,67.59,14.92
2,2,kingisepp,RU,99,1632165191,65,59.3733,28.6134,42.44,9.53
3,3,krasnoarmeysk,RU,100,1632165191,77,51.0239,45.6969,48.70,14.03
4,4,kodiak,US,98,1632164456,83,57.7900,-152.4072,49.39,17.98


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)

locations = city_data_df[['Lat', 'Lng']].astype(float)

weights = city_data_df['Humidity'].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = city_data_df[(city_data_df['Max Temp'] <= 80) & (city_data_df['Max Temp'] > 70) & (city_data_df['Cloudiness']<=25) & (city_data_df['Wind Speed']<=5) & (city_data_df['Humidity'] <=50)]
hotel_df


,City ID,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
38,38,kaputa,ZM,0,1632165203,39,-8.4689,29.6619,78.93,1.66
161,161,cabra,ES,0,1632165257,28,37.4725,-4.4421,78.24,3.24
246,246,busayra,JO,0,1632165298,36,30.7376,35.6078,72.88,3.83
282,282,likasi,CD,0,1632165316,28,-10.9814,26.7333,71.47,3.94
493,493,ukiah,US,1,1632165415,33,39.1502,-123.2078,75.09,3.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = hotel_df.assign(Hotel_Name = '').rename(columns= {'Hotel_Name': 'Hotel Name'})
hotel_df


,City ID,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
38,38,kaputa,ZM,0,1632165203,39,-8.4689,29.6619,78.93,1.66,
161,161,cabra,ES,0,1632165257,28,37.4725,-4.4421,78.24,3.24,
246,246,busayra,JO,0,1632165298,36,30.7376,35.6078,72.88,3.83,
282,282,likasi,CD,0,1632165316,28,-10.9814,26.7333,71.47,3.94,
493,493,ukiah,US,1,1632165415,33,39.1502,-123.2078,75.09,3.44,


In [7]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# location = f"{hotel_df.loc[250, 'Lat']},{hotel_df.loc[250, 'Lng']}"

params = {
    'key': g_key,
    'radius': 5000,
    'keyword': 'hotel',
}

# hotel_response = requests.get(base_url, params).json()

# print(json.dumps(hotel_response, indent = 4))

# print(hotel_response['results'][0]['name'])

for index, row in hotel_df.iterrows():
    
    params['location'] = f"{row['Lat']},{row['Lng']}"    
    
    hotel_response = requests.get(base_url, params).json()
    
    hotel_df.loc[index, 'Hotel Name'] = hotel_response['results'][0]['name']
    
hotel_df


,City ID,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
38,38,kaputa,ZM,0,1632165203,39,-8.4689,29.6619,78.93,1.66,Chelelelwa Lodge
161,161,cabra,ES,0,1632165257,28,37.4725,-4.4421,78.24,3.24,MS Fuente Las Piedras
246,246,busayra,JO,0,1632165298,36,30.7376,35.6078,72.88,3.83,Ein Lahda Camp Site
282,282,likasi,CD,0,1632165316,28,-10.9814,26.7333,71.47,3.94,Hotel De Likasi
493,493,ukiah,US,1,1632165415,33,39.1502,-123.2078,75.09,3.44,Travelodge by Wyndham Ukiah


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations=locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(hotel_marker)

fig

Figure(layout=FigureLayout(height='420px'))